   Test script to load in GFDL FLOR model prediction of sea ice and regrid to a polar stereo projection

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import xarray as xr
import xesmf as xe
import os
# import glob
import seaborn as sns

In [ ]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [ ]:
# Data dir
data_dir = r'/home/disk/sipn/nicway/data/model/nmme/test'
grid_file = r'/home/disk/sipn/nicway/data/grids/GFDLFLOR_gridinfo.nc'
# cfile = 'sic_OImon_GFDL-FLORB01_FLORB01-P1-ECDA-v3.1-062014_r9i1p1_201406-201505.nc'
cfile = 'sic_OImon_GFDL-FLORB01_FLORB01-P1-ECDA-v3.1-041980_r2i1p1_198004-198103.nc'
file_in = os.path.join(data_dir, cfile)

In [ ]:
# Open Data
ds = xr.open_dataset(file_in)
grid = xr.open_dataset(grid_file)

In [ ]:
n_lat = grid.grid_center_lat.values.reshape(tuple(np.flip(grid.grid_dims.T.values,0))) # Reshape
n_lat = np.rad2deg(n_lat)
n_lon = grid.grid_center_lon.values.reshape(tuple(np.flip(grid.grid_dims.T.values,0))) # Reshape
n_lon = np.rad2deg(n_lon)
# print(n_lon.min(), n_lon.max())

In [ ]:
# Fix GFDL grid
# Files comes with incorrect lat/lon at poles. Use correct separte file to update coords
ds.rename({'lat':'nj', 'lon':'ni'}, inplace=True);
ds.coords['nj'] = xr.DataArray(np.arange(0,ds.nj.size,1), dims=('nj'))
ds.coords['ni'] = xr.DataArray(np.arange(0,ds.ni.size,1), dims=('ni'))
ds.coords['lat'] = xr.DataArray(n_lat, dims=('nj','ni'), coords={'nj':ds.nj, 'ni':ds.ni})
ds.coords['lon'] = xr.DataArray(n_lon, dims=('nj','ni'), coords={'nj':ds.nj, 'ni':ds.ni})

In [ ]:
# Average over all sea ice categories 
sic = ds.sic.sum(dim='ct', skipna=False)

In [ ]:
# Get info about target grid
flat = r'/home/disk/sipn/nicway/data/grids/psn25lats_v3.dat'
flon = r'/home/disk/sipn/nicway/data/grids/psn25lons_v3.dat'
NY=304; 
NX=448;

def readBinFile(f, nx, ny):
    with open(f, 'rb') as fid:
        data_array = np.fromfile(f, np.int32)*1e-5
    return data_array.reshape((nx,ny))

In [ ]:
lat = readBinFile(flat, NX, NY).T
lon = readBinFile(flon, NX, NY).T
ds_out = xr.Dataset({'lat': (['x', 'y'],  lat), 'lon': (['x', 'y'], lon)})
ds_out

In [ ]:
# Plot target lat and lon
# ds_out.lon.plot.pcolormesh()

In [ ]:
# ds_out.lat.plot.pcolormesh()

In [ ]:
# sic

In [ ]:
# Calculate regridding matrix
regridder = xe.Regridder(sic, ds_out, 'bilinear', periodic=True)
# Regrid
sic_out = regridder(sic)

In [ ]:
# Define naive_fast that searches for the nearest WRF grid cell center
def naive_fast(latvar,lonvar,lat0,lon0):
    # Read latitude and longitude from file into numpy arrays
    latvals = latvar[:]
    lonvals = lonvar[:]
    dist_sq = (latvals-lat0)**2 + (lonvals-lon0)**2
    minindex_flattened = dist_sq.argmin()  # 1D index of min element
    iy_min,ix_min = np.unravel_index(minindex_flattened, latvals.shape)
    return iy_min,ix_min

In [ ]:
# Testing point selectoin method
# rnd_pts = list(zip(list(np.linspace(sic_out.lat.min().values,sic_out.lat.max().values,50)),
#                    list(np.linspace(sic_out.lon.min().values,sic_out.lon.max().values,50))))
# lat_err = []
# lon_err = []
# for cpt in rnd_pts:
# #     print(cpt)
#     (c_y, c_x) = naive_fast(sic_out.lat.values, sic_out.lon.values, cpt[0], cpt[1])
# #     print(sic_out.isel(x=c_x, y=c_y).lat.values, cpt[0])
# #     print(sic_out.isel(x=c_x, y=c_y).lon.values, cpt[1])
#     lat_err.append((sic_out.isel(x=c_x, y=c_y).lat-cpt[0]).values)
#     lon_err.append((sic_out.isel(x=c_x, y=c_y).lon-cpt[1]).values)
# plt.figure()
# plt.plot(np.linspace(sic_out.lat.min().values,sic_out.lat.max().values,50), lat_err)
# plt.figure()
# plt.plot(np.linspace(sic_out.lon.min().values,sic_out.lon.max().values,50), lon_err)

In [ ]:
# # Test Regridding was successful
# pt_lon = 0
# pt_lat = 85
# (c_y_O, c_x_O) = naive_fast(sic.lat.values, sic.lon.values, pt_lat, pt_lon)
# (c_y_T, c_x_T) = naive_fast(sic_out.lat.values, sic_out.lon.values, pt_lat, pt_lon)
# plt.figure()
# sic.sel(ni=c_x_O, nj=c_y_O).plot(linewidth=4, label='orig projection')  # input data
# sic_out.sel(x=c_x_T, y=c_y_T).plot(label='new projection')# output data
# plt.legend()

In [ ]:
# Set up plotting info
cmap_sic = matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10))
cmap_sic.set_bad(color = 'lightgrey')

In [ ]:
# Plot original projection
plt.figure(figsize=(20,10))
ax1 = plt.axes(projection=ccrs.PlateCarree())
sic.mean(dim='time').plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                 vmin=sic.min().values, vmax=sic.max().values,
                                 cmap=matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10)));
ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
gl = ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
gl.xlabels_bottom = True
gl.ylabels_left = True
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax1.coastlines(linewidth=0.75, color='black', resolution='50m');

In [ ]:
# Plot SIC on target projection
plt.figure(figsize=(10,8));
ax1 = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0))
sic_out.mean(dim='time').plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_sic)
# Add coastlines and meridians/parallels 
ax1.coastlines(linewidth=0.75, color='black', resolution='50m')
ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
ax1.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())
plt.tight_layout()

In [ ]:
# Save new file
f_out = os.path.join(data_dir, os.path.splitext(cfile)[0]+'_Stereo.nc')
sic_out.to_netcdf(f_out)

In [ ]:
# Clean up
regridder.clean_weight_file()  # clean-up